# Spark Dataframe API
**Andrey Titov**
Senior Spark Engineer @ NVIDIA

## На этом занятии
+ Устройство Spark Dataframe API
+ Чтение данных из источника
+ Работа с данными
  - Базовый SQL
  - NA функции
  - Группировки
  - Запись данных
  - Соединения
  - Оконные функции
  - Функции pyspark.sql.functions

In [ ]:
# Файлы с данными
json_file = 'cities.json'
output_parquet_agg = "tmp/agg0.parquet"

## Устройство Spark Dataframe API

**Dataframe:**
+ структурированная колоночная структура данных
+ может быть создана на основе:
  - локальной коллекции
  - файла (файлов)
  - базы данных
+ в python работает значительно быстрее, чем RDD
+ под капотом использует RDD
+ позволяет выполнять произвольные SQL операции с данными
+ аналогично RDD являются ленивыми и неизменяеыми

## Из чего состоит Dataframe
+ схема [pyspsark.sql.StructType](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.StructType)
+ колонки [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column)
+ данные [pyspark.sql.Row](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Row)

In [ ]:
df = spark.read.json(json_file)

In [ ]:
df.printSchema()

In [ ]:
df.show(30, False)

In [ ]:
type(df.schema)

In [ ]:
df.columns

In [ ]:
from pyspark.sql.functions import col
df.select(
    col("continent"), col("country")
).show(3, False)

In [ ]:
df.collect()

## Чтение данных из источника
Основной метод чтения любых источников

```df = spark.read.format(datasource_type).option(datasource_options).load(object_name)```

+ ```datasource_type``` - тип источника ("parquet", "json", "cassandra") и т. д.
+ ```datasource_options``` - опции для работы с источником (логины, пароли, адреса для подключения и т. д.)
+ ```object_name``` - имя таблицы/файла/топика/индекса

[DataframeReader](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader):
+ по умолчанию выводит схему данных
+ является трансформацией (ленивый)
+ возвращает [Dataframe](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)

### Список (неполный) поддерживаемых источников данных
+ Файлы:
  - json
  - text
  - csv
  - orc
  - parquet
+ Базы данных
  - elasticsearch
  - cassandra
  - jdbc
  - hive
+ Брокеры сообщений
  - kafka
  

**Библиотеки для работы с источниками должны быть доступны в JAVA CLASSPATH на драйвере и воркерах!**

## Работа с данными

In [ ]:
df = spark.read.json(json_file)

df.show(20, False)

In [ ]:
# Убираем ненужные колонки
df.drop("_corrupt_record").show(20, False)

In [ ]:
# Удаляем дубликаты
df \
    .drop("_corrupt_record") \
    .distinct() \
    .show()

In [ ]:
# Удаляем пустые строки. Параметр "all" означает, что будут удалены только те строки, в которых ВСЕ элементы null
df \
    .drop("_corrupt_record") \
    .distinct() \
    .na.drop("all") \
    .show()

In [ ]:
# Заполняем пустые значения

df \
    .drop("_corrupt_record") \
    .distinct() \
    .na.drop("all") \
    .na.fill( {'continent': 'Earth', 'population': 0 } ) \
    .show()

In [ ]:
# Пример выборки нескольких колонок

clean_data = df \
    .drop("_corrupt_record") \
    .distinct() \
    .na.drop("all") \
    .na.fill( {'continent': 'Earth', 'population': 0 } ) \
    .select("continent", "country", "name", "population") \

clean_data.show()

In [ ]:
# Строим базовую группировку

clean_data.groupBy('continent').count().show(10, False)

In [ ]:
# Метод count можно спрятать внутри agg()
from pyspark.sql.functions import count

agg = clean_data.groupBy('continent').agg(count("*"))

In [ ]:
# Чтобы колонки имели правильное имя, следует использовать метод alias()

agg = clean_data.groupBy('continent').agg(count("*").alias("count"))
agg.show()

In [ ]:
# Добавим в группировку сумму населения на каждом континенте

from pyspark.sql.functions import count, sum

agg = \
    clean_data \
    .groupBy('continent') \
    .agg(count("*").alias("city_count"), sum('population').alias("population_sum"))


agg.show()

## Запись данных
Основной метод записи в любые системы

```df.write.format(datasource_type).options(datasource_options).mode(savemode).save(object_name)```

+ ```datasource_type``` - тип источника ("parquet", "json", "cassandra") и т. д.
+ ```datasource_options``` - опции для работы с источником (логины, пароли, адреса для подключения и т. д.)
+ ```savemode``` - режим записи данных (добавление, перезапись и т. д.)
+ ```object_name``` - имя таблицы/файла/топика/индекса

[DataFrameWriter](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter):
+ метод ```save``` является действием
+ позволяет работать с партиционированными данными (parquet, orc)
+ не всегда валидирует схему и формат данных


### Список (неполный) поддерживаемых источников данных
+ Файлы:
  - json
  - text
  - csv
  - orc
  - parquet
+ Базы данных
  - elasticsearch
  - cassandra
  - jdbc
  - hive
+ Брокеры сообщений
  - kafka
  

**Библиотеки для работы с источниками должны быть доступны в JAVA CLASSPATH на драйвере и воркерах!**



In [ ]:
# Сохраним данные в parquet, предварительно отфильтровав данные

condition = col("continent") != "Earth"

agg \
    .filter(condition) \
    .write \
    .format("parquet") \
    .mode("overwrite") \
    .save(output_parquet_agg)

print("Ok! Data is written to {}".format(output_parquet_agg))

In [ ]:
# P.S.
# Когда мы делаем .filter в DataFrame API, мы передаем условие типа pyspark.sql.column.Column.
print(type(condition))

# когда раньше мы использовали лямбда функции в RDD, мы передавали лямбда функцию:
condition_old = lambda x: x != "Earth"
print(type(condition_old))

## Соединения

Join'ы позволяют соединять два DF в один по заданным условиям.

По типу условия join'ы делятся на:
+ equ-join - соединение по равенству одного или более ключей
+ non-equ join - соединение по условию, отличному от равенства одного или более ключей

По методу соединения join'ы бывают:
![Joins](http://kirillpavlov.com/images/join-types.png)
[Источник](http://kirillpavlov.com/blog/2016/04/23/beyond-traditional-join-with-apache-spark/)

При выполнении join Spark автоматически выбирает один [из доступных алгоритмов](https://youtu.be/fp53QhSfQcI) соединения и не всегда делает это оптимально, часто применяя cross join. Поэтому, в последних версиях Spark метод ```join()``` приведет к ошибке, если под капотом он будет использовать cross join. Отключить эту проверку можно с помощью опции ```--conf spark.sql.crossJoin.enabled=true```

In [ ]:
# Для демонстрации работы join используем подгтовленные данные
clean_data.printSchema()

agg = spark.read.parquet(output_parquet_agg)
agg.printSchema()

In [ ]:
# Самый простой join - inner join по равенству одной колонки
joined = clean_data.join(agg, 'continent', 'inner')

joined.printSchema()

joined.show()

In [ ]:
# Inner join по равенству двух колонок. Поскольку двух одинаковых колонок у нас нет, мы создадим их из константы
from pyspark.sql.functions import lit

left = clean_data.withColumn("x", lit("x"))
right = agg.withColumn("x", lit("x"))

joined = left.join(right, ['continent', 'x'], 'inner')

joined.printSchema()

joined.show()

In [ ]:
# non-equ left join
from pyspark.sql.functions import lit

left = clean_data.withColumn("city_count_max", lit(2)).withColumnRenamed("continent", "continent_left")
right = agg.withColumnRenamed("continent", "continent_right")

join_condition = (col("continent_left") == col("continent_right")) & (col("city_count") < col("city_count_max"))

joined = left.join(right, join_condition, 'left')

joined.printSchema()

joined.show()

In [ ]:
# non-equ right join
from pyspark.sql.functions import lit

left = clean_data.withColumn("city_count_max", lit(2)).withColumnRenamed("continent", "continent_left")
right = agg.withColumnRenamed("continent", "continent_right")

join_condition = (col("continent_left") == col("continent_right")) & (col("city_count") < col("city_count_max"))

joined = left.join(right, join_condition, 'right')

joined.printSchema()

joined.show()

In [ ]:
# cross join
clean_data.crossJoin(agg).show(30, False)

In [ ]:
# Один из самых простов вариантов ускорить работу join - сделать broadcast.
# При этом DF будет целиком склонирован на каждый воркер, что минимизирует shuffle во время выполнения join'а

from pyspark.sql.functions import broadcast

joined = clean_data.join(broadcast(agg), 'continent', 'inner')

joined.printSchema()

joined.show()

## Оконные функции

Оконные функции позволяют делать функции над "окнами" (кто бы мог подумать) данных

Окно создается из класса [pyspark.sql.Window](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Window) с указанием полей, определяющих границы окон и полей, определяющих порядок сортировки внутри окна:

```window = Window.partitionBy("a", "b").orderBy("a")```

Применяя окна, можно использовать такие полезные функции из [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions), как ```lag()``` и ```lead()```, а также эффективно работать с данными time-series, вычисляя такие параметры, как, например, среднее значение заданного поля за 3-х часовой интервал

In [ ]:
# В нашем случае, используя оконные функции, мы можем построить DF из предыдущих примеров c join, 
# но без использования соединения

from pyspark.sql import Window
import pyspark.sql.functions as F

window = Window.partitionBy("continent")

agg = clean_data \
    .withColumn("city_count", F.count("*").over(window)) \
    .withColumn("population_sum", F.sum("population").over(window)) \

agg.show()

## Функции pyspark.sql.functions

Spark обладает достаточно большим набором встроенных функций, доступных в [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions), поэтому перед тем, как писать свою UDF, стоит обязательно поискать нужную функцию в данном пакете.

К тому же, все функции Spark принимают на вход и возвращают [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column), а это значит, что вы можете совмещать функции вместе

**Также важно помнить, что функции и колонки в Spark могут быть созданы без привязки к конкретным данным и DF**

In [ ]:
from pyspark.sql.functions import *

avg_pop = \
    to_json(
        struct(
            (col("population_sum") / col("city_count")).alias("value")
        )
    ).alias("avg_pop")

agg.select(col("*"), avg_pop).show()


In [ ]:
# Большим преимуществом Spark по сравнению с большинством SQL ориентированных БД является наличие
# встроенных функций работы со списками, словарями и структурами данных

from pyspark.sql.functions import *

all_in_one = agg.select(struct(*agg.columns).alias("allinone"))

all_in_one.printSchema()
all_in_one.show(20, False)

In [ ]:
# Например, можно создавать массивы и объединять их

from pyspark.sql.functions import *

arrays = \
    spark.range(0,1) \
    .withColumn("a", array(lit(1), lit(2), lit(3))) \
    .withColumn("b", array(lit(4),lit(5),lit(6))) \
    .select(array_union(col("a"), col("b")).alias("c"))


arrays.show(1, False)

Также, в разделе [SQL, Built-in Functions](https://spark.apache.org/docs/latest/api/sql/index.html) присутствует еще более широкий список функций, доступных в Spark. Некоторые из них отсутствуют в [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions)! 

Эти функции нельзя использовать как обычные методы над [pyspark.sql.Column](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column), однако вы можете использовать метод ```expr()``` для этого.

In [ ]:
# В данном примере мы используем Java функцию с помощью функции java_method
# Запомните этот пример и используйте всегда, когда вам не хватает какой-либо функции в pyspark, 
# доступной в Java, ведь, используя такой подход, вы не снижаете производительность вашей программы за счет
# передачи данных между Python и JVM приложением Spark, и при этом вам не нужно уметь писать код на Java/Scala :)

from pyspark.sql.functions import *

spark.range(0,1).withColumn("a", expr("java_method('java.util.UUID', 'randomUUID')")).show(1, False)

## Выводы
**Dataframe API**:
+ мощный инструмент для работы с данными
+ в отличие от RDD, Dataframe API устроен так, что все вычисления происходят в JVM
+ обладает единым API для работы с различными источниками данных
+ имеет большой набор встроенных функций работы с данными
+ имеет возможность использовать в pyspark функции, доступные в Java

# Спасибо!